Install dependencies

In [ ]:
pip install --upgrade "langchain>=0.1.0" "pydantic>=2.0.0" rank_bm25 PyPDF2 ragas bert-score pandas

Prepare the models

In [2]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai.chat_models.azure import AzureChatOpenAI
import json
from langchain_core.documents import Document
from rank_bm25 import BM25Okapi
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
import os



llm_model = AzureChatOpenAI(
    azure_deployment="gpt-4.1",
    model_version="2025-04-14",
    api_key="*******",
    api_version="2024-12-01-preview",
    azure_endpoint="*********",
    temperature=0
)


embeddings = AzureOpenAIEmbeddings( 
    api_key="******",
    azure_endpoint="**************",
    openai_api_version="2023-05-15",
    dimensions=1024
   )
vector_store = InMemoryVectorStore(embeddings)
full_documents = []

Convert PDF into text

In [ ]:
import json
from PyPDF2 import PdfReader
from tqdm import tqdm

def pdf_to_chunks_json(
    pdf_path: str,
    file_url: str,
    id: str,
    output_json: str = None
):
    reader = PdfReader(pdf_path)
    content = []
    for page_num, page in enumerate(tqdm(reader.pages, desc="Reading PDF")):
        text = page.extract_text() or ""
        if text.strip():
            content.append({
                "chunks": [{
                    "text": text.strip(),
                    "metadata": {"page": page_num + 1}
                }]
            })

    result = [{
        "file": file_url,
        "id": id,
        "content": content
    }]

    if output_json:
        with open(output_json, "w", encoding="utf-8") as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
    return result

# Example usage:
pdf_to_chunks_json(
    pdf_path="pdf/memoria22.pdf",
    file_url="https://cuacfm.org",
    id="memoria22",
    output_json="chunks/memoria22.json"
)

Store chunks into vector DB

In [ ]:

# Load the JSON
with open('chunks/memoria22.json', 'r') as f:
    data = json.load(f)

chunks = []
texts = []
for entry in data:
    for content in entry.get('content', []):
        for chunk in content.get('chunks', []):
            text = chunk['text']
            metadata = chunk.get('metadata', {})
            chunks.append((text, metadata))
            texts.append(text)

# Prepare BM25
tokenized_corpus = [text.split() for text in texts]
bm25 = BM25Okapi(tokenized_corpus)

# Define the prompt template for the chain
template = """
<document>
{doc_content}
</document>

Here is the chunk we want to situate within the whole document
<chunk>
{chunk_content}
</chunk>

Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk.
Answer only with the succinct context and nothing else.
"""
prompt = PromptTemplate(
    input_variables=["doc_content", "chunk_content"],
    template=template
)

chain = prompt | llm_model

doc_content = "\n".join(texts)
documents = []

for i, (text, metadata) in enumerate(chunks):
    # Run the chain to get the context    
    context = chain.invoke({"doc_content": doc_content, "chunk_content": text})
    print(context)
    if hasattr(context, "content"):
        context = context.content
    metadata['situated_context'] = context
    metadata['original_context'] = chunk
    scores = bm25.get_scores(text.split())
    metadata['bm25_score'] = float(scores[i])
    document = Document(page_content=text, metadata=metadata)
    documents.append(document)
    full_documents.append(document)

vector_store.add_documents(documents=documents)


Print vectors

In [ ]:
top_n = 100
for index, (id, doc) in enumerate(vector_store.store.items()):
    if index < top_n:
        # docs have keys 'id', 'vector', 'text', 'metadata'
        print(f"{doc['metadata']} --> {doc['metadata']}")
    else:
        break   

Prepare the model to answer questions

In [44]:
# Define the prompt template for the chain
template = """
You are a helpful assistant that can answer questions about CUAC FM. Here is the context:
{context} and here the question:
{query}
Mention the page number and name of the document of the context where the answer is found at the end of the answer.
"""
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=template
)
chain = prompt | llm_model

Prepare retrievers: bm25, vector and hybrid

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

def retriever_hybrid(query):
    # BM25 retriever
    bm25_retriever = BM25Retriever.from_documents(full_documents)
    bm25_retriever.k = 20

    # Vector retriever from in-memory vector store
    vector_retriever = vector_store.as_retriever(search_kwargs={"k": 20},search_type="similarity")
    # Ensemble retriever
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, vector_retriever],
        weights=[0.4, 0.6]
    )
    hybrid_result = ensemble_retriever.get_relevant_documents(query)
    return hybrid_result

    
def retriever_bm25(query):
    bm25_retriever = BM25Retriever.from_documents(full_documents)
    bm25_retriever.k = 20
    return bm25_retriever.get_relevant_documents(query)

def retriever_vector(query):
    vector_retriever = vector_store.as_retriever(search_kwargs={"k": 20},search_type="similarity")
    return vector_retriever.get_relevant_documents(query)

def pretty_print_documents(docs, max_docs=5):
    for i, doc in enumerate(docs[:max_docs]):
        print(f"\nResult {i+1}:")
        print(f"Text: {doc.page_content[:300]}...")  # Print first 300 chars
        print(f"Metadata: {doc.metadata}")


query = "¿Qué es CUAC FM?"

result = chain.invoke({"context": retriever_hybrid(query), "query": query})
print(result.content)



Prepare data frame to evaluate with RAGAS

In [ ]:
import pandas as pd
# Example query
cuac_fm_questions = [
    # 📻 Preguntas generales sobre CUAC FM
    "¿Qué es CUAC FM?",
    "¿Cuándo y cómo nació CUAC FM?",
    "¿Qué tipo de radio es CUAC FM?",
    "¿Dónde tiene su sede CUAC FM?",
    "¿Qué significa CUAC?",

    # 🧑‍🤝‍🧑 Participación y organización
    "¿Quién puede participar en CUAC FM?",
    "¿Cómo se organizan los programas en CUAC FM?",
    "¿Qué papel tiene el voluntariado en CUAC FM?",
    "¿Qué formación ofrece CUAC FM a los nuevos colaboradores?",
    "¿Cómo se puede proponer un nuevo programa en CUAC FM?",

    # 🗣️ Contenido y programación
    "¿Qué tipo de programas se emiten en CUAC FM?",
    "¿Qué temas suelen tratarse en la emisora?",
    "¿Cómo se selecciona la programación de CUAC FM?",
    "¿Hay espacio para la música local y emergente en CUAC FM?",
    "¿CUAC FM tiene una programación estable o cambia a menudo?",

    # 🎧 Tecnología y difusión
    "¿En qué frecuencia emite CUAC FM?",
    "¿Se puede escuchar CUAC FM por internet?",
    "¿Qué tipo de tecnología usa CUAC FM para grabar y emitir programas?",
    "¿Los programas están disponibles en formato podcast?",
    "¿Dónde se pueden encontrar los programas antiguos?",

    # 📢 Compromiso social y legal
    "¿Qué papel juega CUAC FM como medio comunitario?",
    "¿Qué derechos y deberes tiene CUAC FM según la legislación española?",
    "¿CUAC FM ha tenido conflictos o problemas legales por su licencia de emisión?",
    "¿Cómo se financia CUAC FM?",
    "¿Qué relación tiene CUAC FM con la Red de Medios Comunitarios (ReMC)?",

    # 🌍 Impacto y proyección
    "¿Qué impacto tiene CUAC FM en la comunidad local?",
    "¿Qué actividades realiza CUAC FM fuera de la radio (talleres, eventos, colaboraciones)?",
    "¿Ha recibido CUAC FM algún reconocimiento o premio?",
    "¿Con qué otras radios o entidades colabora CUAC FM?",
    "¿Qué desafíos enfrenta actualmente CUAC FM?"
]


cuac_fm_answers = [
    # 📻 Preguntas generales sobre CUAC FM
    "CUAC FM es una emisora comunitaria sin ánimo de lucro que emite desde A Coruña.",
    "Nació en 1996 como una iniciativa de estudiantes universitarios de la Universidade da Coruña.",
    "Es una radio libre, comunitaria y educativa, gestionada por voluntariado.",
    "Tiene su sede en A Coruña, Galicia, España.",
    "CUAC significa 'Colectivo Universitario de Actividades Culturales'.",

    # 🧑‍🤝‍🧑 Participación y organización
    "Cualquier persona con interés en la comunicación y en aportar contenido social o cultural puede participar.",
    "Cada programa es gestionado por su propio equipo, y CUAC proporciona el espacio, formación y emisión.",
    "El voluntariado es fundamental: todas las tareas de programación, técnica y gestión son realizadas por voluntarios.",
    "CUAC ofrece talleres de formación técnica y de comunicación radiofónica a sus nuevos miembros.",
    "Basta con presentar una propuesta al equipo de coordinación y seguir unas pautas básicas para comenzar a emitir.",

    # 🗣️ Contenido y programación
    "Emiten programas de música, cultura, sociedad, política, feminismo, ecología, etc.",
    "Tratan temas sociales, culturales, educativos y comunitarios, especialmente los que no tienen cabida en medios comerciales.",
    "La programación se construye de forma colaborativa entre los diferentes programas y el equipo de coordinación.",
    "Sí, CUAC da mucha visibilidad a artistas y grupos locales y emergentes.",
    "Tiene una base estable de programas, pero también permite rotación y nuevos proyectos cada temporada.",

    # 🎧 Tecnología y difusión
    "Emite en el 103.4 FM en A Coruña.",
    "Sí, también puede escucharse en línea desde su página web y apps de radio por internet.",
    "Utiliza equipos de grabación digital, software de automatización y plataformas de streaming.",
    "Sí, muchos programas están disponibles como podcast en su web y en plataformas como iVoox o Spotify.",
    "En la página oficial de CUAC FM y en sus perfiles de podcasting.",

    # 📢 Compromiso social y legal
    "Es un medio al servicio de la comunidad, ofreciendo una voz a colectivos y personas sin representación en medios tradicionales.",
    "Según la legislación española, tiene derecho a emitir como medio comunitario, aunque la normativa aún es limitada.",
    "Sí, CUAC FM ha denunciado en varias ocasiones la falta de reconocimiento legal y ha sufrido amenazas de cierre por falta de licencia.",
    "Se financia a través de subvenciones, colaboraciones públicas, donaciones y autofinanciación.",
    "CUAC FM es miembro fundador de la Red de Medios Comunitarios (ReMC), con la que colabora activamente en iniciativas de defensa de los medios libres.",

    # 🌍 Impacto y proyección
    "Tiene un papel activo en la comunidad coruñesa, dando voz a causas sociales, iniciativas culturales y movimientos ciudadanos.",
    "Organiza talleres, jornadas, encuentros de radios libres y actividades educativas en colegios e institutos.",
    "Sí, ha recibido reconocimientos por su trayectoria y por la promoción de la comunicación libre y democrática.",
    "Colabora con radios libres de toda España y Europa, así como con universidades, ONGs y colectivos sociales.",
    "Enfrenta desafíos como la falta de apoyo institucional, el acceso a licencias legales y la sostenibilidad económica a largo plazo."
]

results_hybrid = []
results_bm25 = []
results_vector = []
llm_answer_hybrid = []
llm_answer_bm25 = []
llm_answer_vector = []

for question in cuac_fm_questions:
    hybrid_result = retriever_hybrid(question)
    bm25_result = retriever_bm25(question)
    vector_result = retriever_vector(question)
    results_hybrid.append(hybrid_result)
    results_bm25.append(bm25_result)
    results_vector.append(vector_result)
    llm_answer_hybrid.append(chain.invoke({"context": hybrid_result, "query": question}))
    llm_answer_bm25.append(chain.invoke({"context": bm25_result, "query": question}))
    llm_answer_vector.append(chain.invoke({"context": vector_result, "query": question}))

results = []
index = 0
for question in cuac_fm_questions:
        print(index)
        results.append({
                "question": question,
                "retrieval_type": "hybrid",
                "context": results_hybrid[index],
                "answer": llm_answer_hybrid[index],
                "ground_truth": cuac_fm_answers[index]
            })
        results.append({
                "question": question,
                "retrieval_type": "bm25",
                "context": results_bm25[index],
                "answer": llm_answer_bm25[index],
                "ground_truth": cuac_fm_answers[index]
            })
        results.append({
                "question": question,
                "retrieval_type": "vector",
                "context": results_vector[index],
                "answer": llm_answer_vector[index],
                "ground_truth": cuac_fm_answers[index]
            })
        index += 1

df = pd.DataFrame(results)

print(df)


Run RAGAS evaluation

In [ ]:
import os
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from bert_score import score as bert_score
from ragas.evaluation import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas.evaluation import EvaluationDataset

os.environ["OPENAI_API_KEY"] = "sk-proj----"

print(df.columns)

dataset = EvaluationDataset.from_pandas(df)

# 6. Evaluate with RAGAS
results_evaluation = evaluate(
    dataset,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
)





Store data frame into CSV

In [43]:
df.to_csv('output.csv', index=False)

Evaluate the result

In [ ]:
print(results_evaluation)

# 7. Add RAGAS results_evaluation to DataFrame
# Handle both dict and list output
if hasattr(results_evaluation, 'scores'):
    scores = results_evaluation.scores
    if isinstance(scores, dict):
        for metric_name, metric_scores in scores.items():
            df[metric_name] = metric_scores
    elif isinstance(scores, list):
        # Try to convert to DataFrame and concat
        scores_df = pd.DataFrame(scores)
        df = pd.concat([df, scores_df], axis=1)

# 8. Evaluate with BERTScore
if 'user_input' in df.columns and 'reference' in df.columns:
    P, R, F1 = bert_score(df["user_input"].astype(str).tolist(), df["reference"].astype(str).tolist(), lang="es")
    df["bertscore_f1"] = F1.tolist()

# 9. Show all metrics per question and retrieval type (if present)
cols_to_show = [c for c in ["user_input", "retrieval_type", "faithfulness", "answer_relevancy", "context_precision", "context_recall", "bertscore_f1"] if c in df.columns]
print(df) 

Store evaluation into CSVs

In [ ]:
import pandas as pd

df_bm25 = pd.read_csv('output_bm25.csv')
df_vector = pd.read_csv('output_vector.csv')
df_hybrid = pd.read_csv('output_hybrid.csv')

metrics = ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall', 'bertscore_f1']

print("BM25 Means:")
print(df_bm25[metrics].mean())
print("\nVector Means:")
print(df_vector[metrics].mean())
print("\nHybrid Means:")
print(df_hybrid[metrics].mean())